Introducción a TensorBoard
===

Esta lección presenta una introducción al uso de TensorBoard para visualizar las gráficas de flujos de datos, la construcción de ámbitos y el uso de resumenes de cómputos. El lector puede referirse al sitio de oficial de TensorFlow (https://www.tensorflow.org) para mayor información y para obtener una visión más profunda de las herramientas y metodologías disponibles.

## Visualización de flujos de datos en TensorBoard

`TensorBoard` es una herramienta que permite visualizar las gráficas de flujo de datos generadas por TensorFlow. A continuación se construye la gráfica para la expresión `5 * 2 - (2 + 3)`.

In [11]:
## Se crea una nueva gráfica de flujo de datos
g = tf.Graph()

with g.as_default():
    
    ## Con la nueva gráfica por defecto, se construyen los nodos 
    node1 = tf.constant(5.0, name='const_5')  
    node2 = tf.constant(2.0, name='const_2')  
    node3 = tf.constant(3.0, name='const_3')  
    node4 = tf.multiply(node1, node2, name='node_4')
    node5 = tf.add(node3, node2, name='node_5')
    node6 = tf.subtract(node4, node5, name='node_6')
    
    ## Se usa FileWriter para genera la información del grafo.
    tf.summary.FileWriter('/tmp/my_graph', g)

Para visualizar el grafo, puede usar una de las siguientes alternativas:

* Lanzar `TensorBoard` desde el Terminal usando:

```
       tensorboard --logdir="/tmp/my_graph"
```     
     
  Abra el explorador de internet en la dirección http://localhost:6006
    
* Abrir la extensión TensorBoards en la pestaña lateral de Jupyter Lab.

**Nota:** Si desea escribir la gráfica al directorio actual puede usar

```
    tf.summary.FileWriter('.', g)
```
    
y para abrirla en TensorBoard

```
    tensorboard --logdir .
```

El gráfico generado es el siguiente:

![TensorBoardIntro-01.png](assets/TensorBoardIntro-01.png)

**Actividad.--** Visualice el flujo de datos de la expresión

$$-0.9 * 2^3 + 2.5 * 2^2 - 3.8 * 2 -4.3$$

usando `TensorBoard`.

## División de una gráfica de flujo de datos en ámbitos

Es posible crear subgráficas dentro de una gráfica principal usando ámbitos. En este caso, se crea el ámbito y se crean los nodos dentro de él. Por ejemplo:

In [12]:
## crea una nueva gráfica
g = tf.Graph()

## crea los nodos dentro de la grafica g 
with g.as_default():
    
    ##  Se crea el ámbito A con tres nodos    
    with tf.name_scope('A'):
        a = tf.add(1, 2, name='Aa')
        b = tf.add(3, 4, name='Ab')
        c = tf.multiply(a, b, name='Ac')

    ##  Se crea el ámbito B con tres nodos    
    with tf.name_scope('B'):
        d = tf.add(5, 6, name='Bd')
        e = tf.add(7, 8, name='Be')
        f = tf.multiply(d, e, name='Bf')

    ## Este es el nodo raíz de los dos ámbitos
    h = tf.subtract(c, f, 'raiz')

    ## Escribe la gráfica de flujo de datos al disco
    tf.summary.FileWriter('/tmp/ambito', g)

Para abrir la gráfica, use el comando `tensorboard --logir=/tmp/ambito` en términal y luego habra la dirección http://localhost/6006 en el explorador. 

La gráfica generada es la siguiente:

![TensorBoardIntro-02.png](assets/TensorBoardIntro-02.png)

donde la parte superior corresponde a la grafica colapsada y la parte inferior a la gráfica expandida.

## Resumenes de ejecuciones en TensorBoard

En el siguiente ejemplo, se ejemplifica como organizar en ámbitos el código en un programa de TensorFlow, la forma de construir resumenes, y el reporte en `TensorBoard`. En este ejemplo, se computa $y=f(x)$ para diferentes valores de $x$ y $\sum f(x)$.

In [41]:
## Borra el directorio de trabajo en cada corrida
!rm -rf /tmp/example

## Crea una gráfica
graph = tf.Graph()

with graph.as_default():
    
    ## Se definen las variables a usar en la corrida
    with tf.name_scope('variables'):
        num_corridas = tf.Variable(0, dtype=tf.int32, name='num_corridas')
        suma_y = tf.Variable(0.0, dtype=tf.float32, name='suma_y')
        
    ## Se definen las variables que modifica el usuario
    with tf.name_scope('x'):
        x = tf.placeholder(tf.float32, shape=[], name='placeholder_x')
        
    ## Cómputos (evalua f(x))
    with tf.name_scope('fx'):
        y = tf.square(x, name='y')
        
    ## Almacena los contadores
    with tf.name_scope('contadores'):
        incr_suma_y = suma_y.assign_add(y)
        incr_num_corridas = num_corridas.assign_add(1)
        
    ## Genera los resumenes a partir de las variables
    with tf.name_scope('resumenes'):
        tf.summary.scalar('Y', y)
        tf.summary.scalar('Suma_y', suma_y)
        
    ## Inicializa las variables y los resumenes
    with tf.name_scope('operaciones'):
        init = tf.global_variables_initializer()
        resumenes = tf.summary.merge_all()
        
        
## Ejecución
##   Crea una sesión para realizar los cómputos
sess = tf.Session(graph=graph)

##   Crea el directorio donde se almacenarán los resultados
writer = tf.summary.FileWriter('/tmp/example', graph)

##   Inicializa las variables
sess.run(init)

## Esa función encapsula la corrida para un valor z
def f(z):
    ## Realiza el cómputo
    _, step, summary = sess.run([y,               # Computa y = f(x)
                                 incr_suma_y,     # sumatoria de f(x)
                                 resumenes],      # Genera el resumen (Y, Suma_Y)
                                feed_dict={x:z})  # Pasa el valor de x a TensorFlow
    
    ## Almacena el resumen en disco duro
    writer.add_summary(summary, global_step=step)
        
##
## Se llama a la función f(x) para x = 0, 1, 2, ...
##
for a in np.arange(0, 10, 1, np.float32):
    f(a)

## se vacia el buffer al disco duro y se cierra la conexión
writer.flush()
writer.close()

## Se cierra la sesión
sess.close()

La organización del código en ámbitos permite su mejor organización. En la siguiente figura se muestra la gráfica computacional generada.

![TensorBoardIntro-03.png](assets/TensorBoardIntro-03.png)

Los gráficos generados por los resumenes son los siguientes:

![TensorBoardIntro-04.png](assets/TensorBoardIntro-04.png)